## DATA DEFINITION


In [32]:
import pandas as pd
import sys
import os
import nibabel as nib
import torch 
from monai import transforms

sys.path.append(os.path.abspath("/scratch/ml9715/DINO-3D/base_modules"))
from src.models.vit import ViT

TRAIN_DATASET_CSV = '/scratch/by2026/BrainATLAS/mae/mae/data_csv/mae_UKB_ADNI_HCP_CamCAN_IXI_train_with_stat.csv'
VALIDATION_DATASET_CSV = '/scratch/by2026/BrainATLAS/mae/mae/data_csv/mae_UKB_ADNI_HCP_CamCAN_IXI_val_with_stat.csv'
MODEL_CHECKPOINT_PATH = '/scratch/ml9715/DINO-3D/checkpoints/RUN_1/checkpoint0050.pth'
FEATURES_TO_VALIDATE = ['age', 'gender']




validation_dataset_df = pd.read_csv(VALIDATION_DATASET_CSV).head(1000)

len(validation_dataset_df)

/tmp/ipykernel_446062/4034038160.py:19: DtypeWarning: Columns (2,3,5,7,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  validation_dataset_df = pd.read_csv(VALIDATION_DATASET_CSV).head(1000)


1000

## LOADING ViT WITH PRE-TRAINED WEIGHTS

In [28]:
# Load the CSV and model checkpoint

checkpoint = torch.load(MODEL_CHECKPOINT_PATH, map_location="cpu")

# Define the ViT model and load the checkpoint
vit_model = ViT(in_chans=1, img_size=(96, 96, 96), patch_size=(12, 12, 12), 
                patch_embed='conv', pos_embed='sincos', use_flash_attn=True)
vit_state_dict = {k.replace("model.", ""): v for k, v in checkpoint['student'].items() if "model." in k}
vit_model.load_state_dict(vit_state_dict, strict=False)
vit_model.eval()

ViT(
  (patch_embedding): PatchEmbeddingBlock(
    (patch_embeddings): Conv3d(1, 768, kernel_size=(12, 12, 12), stride=(12, 12, 12))
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (blocks): ModuleList(
    (0-11): 12 x FlashAttentionBlock(
      (mlp): MLPBlock(
        (linear1): Linear(in_features=768, out_features=3072, bias=True)
        (linear2): Linear(in_features=3072, out_features=768, bias=True)
        (fn): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): SABlock(
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (qkv): Linear(in_features=768, out_features=2304, bias=False)
        (drop_output): Dropout(p=0.0, inplace=False)
        (drop_weights): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
  (norm): LayerNo

## EXTRACTING EMBEDDINGS AND CREATING NEW DATAFRAME


In [33]:
def preprocess_image(img_path):
    img = nib.load(img_path).get_fdata()
    img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
    if img.shape[-3:] != (96, 96, 96):
        print('Error: Incorrect image size')
        img = transforms.Resize(spatial_size = (96, 96, 96))(img)
    return img

# Prepare a list to collect data for the new DataFrame
data_list = []

# Iterate through the CSV to extract features and collect selected columns
for _, row in validation_dataset_df.iterrows():
    if row['is_img_path_T1_mni152']:
        img_path = row['img_path_T1_mni152']
        
        try:
            # Preprocess the image
            img_tensor = preprocess_image(img_path)
            with torch.no_grad():
                features = vit_model(img_tensor.unsqueeze(0))  # Get features
    
            # Collect the features and selected columns
            data_entry = {
                'features': features.squeeze().numpy(),  # Convert tensor to numpy array for storage
            }
            # Add selected columns
            for col in FEATURES_TO_VALIDATE:
                data_entry[col] = row[col]
            
            data_list.append(data_entry)
        
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# Create a new DataFrame with features and selected columns
features_df = pd.DataFrame(data_list)
print(features_df.head())

Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect image size
Error: Incorrect ima

In [35]:
print(X_train)

[[ 1.1218566  -0.28085098  1.9330194  ... -0.35039976 -1.0436895
  -0.99394745]
 [ 0.82916003 -0.12820335  1.7916157  ... -0.16835319 -0.56006485
  -0.75818783]
 [ 1.1311816   0.02959479  1.8058372  ... -0.35044867 -0.74237186
  -0.70605123]
 ...
 [ 1.2728965  -0.02725515  1.924251   ... -0.29182583 -0.9546871
  -0.7392741 ]
 [ 1.1831354  -0.18322287  1.906201   ... -0.20106746 -1.0703359
  -0.8113101 ]
 [ 0.98301053 -0.24681358  1.8782861  ... -0.25532988 -0.77152073
  -0.85178787]]


In [36]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score
import numpy as np

# Extract features, age, and gender

features_df = features_df.dropna(subset=['age'])
X = np.array(features_df['features'].tolist())  # Convert list of features to numpy array
y_age = features_df['age'].values
y_gender = features_df['gender'].values

# Split the data into training and testing sets
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_age, y_gender, test_size=0.2, random_state=42
)

# Train linear regression model for age prediction
age_model = LinearRegression()
age_model.fit(X_train, y_age_train)

# Predict age and evaluate with R^2
y_age_pred = age_model.predict(X_test)
age_r2 = r2_score(y_age_test, y_age_pred)
print("R^2 Score for Age Prediction:", age_r2)

# Train logistic regression model for gender classification
gender_model = LogisticRegression(max_iter=1000)
gender_model.fit(X_train, y_gender_train)

# Predict gender and evaluate accuracy
y_gender_pred = gender_model.predict(X_test)
gender_accuracy = accuracy_score(y_gender_test, y_gender_pred)
print("Accuracy for Gender Prediction:", gender_accuracy)


R^2 Score for Age Prediction: -2.0814279247004346
Accuracy for Gender Prediction: 0.8857142857142857
